In [1]:
#TODO 把feature加入LSTM
#方法1.input时多加两个feature,makeFeatureVec后面加,可解释性差
#方法2.先训练, 不更改, 找到最优参数后, 把最后一个hidden layer输出的vector 再加上additional feature 得到新的feature vector,
#再去训练一个multiple logistic regression
#方法3, 改模型结构, 增加一个input
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import re
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

import language_check

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pyLDAvis
from pyLDAvis.sklearn import prepare

plt.style.use('seaborn-colorblind')

# Setup Pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_colwidth', 100)

pyLDAvis.enable_notebook()
# plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi']= 100

import warnings
warnings.simplefilter("ignore")

In [2]:
training_set  = pd.read_csv('training_set_rel3.tsv', sep='\t', encoding = "ISO-8859-1")\
            .rename(columns={'essay_set': 'topic', 'domain1_score': 'target_score', 'domain2_score': 'topic2_target'})
count_vector  = pd.read_csv('count_vector.tsv', sep='\t', encoding = "ISO-8859-1")
count_vector=count_vector.drop(labels = "Unnamed: 0", axis = 1)
training_set.sample()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,essay_id,topic,essay,rater1_domain1,rater2_domain1,rater3_domain1,target_score,rater1_domain2,rater2_domain2,topic2_target,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,err_ratio,avg_len,total_err_ratio,avg_wdlen,count_vector
6760,6760,6760,6760,6760,6760,10319,4,"In the story Winter Hibiscus by: Minfong Ho, a girl named Saeng is try to adjust to @LOCATION1...",2,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.054878048780487805, 0.018292682926829267)",13.666667,0.073171,4.219512,"(0, 31237)\t1\n (0, 6806)\t1\n (0, 21346)\t1\n (0, 36563)\t1\n (0, 16144)\t2\n (0, 29110)..."


In [5]:
pca5 = PCA(n_components=5)

In [6]:
principalComponents5 = pca5.fit_transform(count_vector)

In [7]:
principalDf = pd.DataFrame(data = principalComponents5
             , columns = ['w1', 'w2','w3','w4','w5',])

## Default model

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

In [9]:
X = pd.concat([principalDf, training_set[['avg_len','total_err_ratio','avg_wdlen']]], axis = 1)

In [10]:
X.to_csv('features_multiclass.tsv', sep='\t', encoding = "ISO-8859-1")

In [14]:
y = training_set[['target_score']]

In [15]:
default_model = LogisticRegression()

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
default_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
y_pred=default_model.predict(X_test)
r1= accuracy_score(y_test, y_pred)
r2 = cohen_kappa_score(y_test,y_pred,weights='quadratic')
#TODO:check QWK
print("accuracy_score: ", r1)
print("QWK: ", r2)

accuracy_score:  0.449537750385208
QWK:  0.6825070823310198


## RandomizedSearchCV

In [19]:
%%time
parameters = {
              "tol":[1e-4,1e-3,1e-2,1e-1,1e-5,1e-6], 
              "max_iter":[x*10 for x in range(1,11,1)]}
clf = RandomizedSearchCV(LogisticRegression(), param_distributions=parameters, n_jobs=4 ,n_iter=10, random_state=0)
clf.fit(X=X_train, y=y_train)
model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

0.4393063583815029 {'tol': 1e-06, 'max_iter': 60}
Wall time: 15.3 s


In [20]:
y_pred=model.predict(X_test)
r1= accuracy_score(y_test, y_pred)
r2 = cohen_kappa_score(y_test,y_pred,weights='quadratic')
#TODO:check QWK
print("accuracy_score: ", r1)
print("QWK: ", r2)

accuracy_score:  0.4503081664098613
QWK:  0.6824985819106414


# GridSearchCV

In [21]:
%%time
parameters = {"tol":[1e-4,1e-3,1e-2,1e-1,1e-5,1e-6], 
              "max_iter":[x*10 for x in range(1,11,1)]}
clf = GridSearchCV(LogisticRegression(), param_grid=parameters, n_jobs=6)
clf.fit(X=X_train, y=y_train)
model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

0.4394990366088632 {'max_iter': 20, 'tol': 0.0001}
Wall time: 51.9 s


In [22]:
y_pred=model.predict(X_test)
accuracy_score(y_test, y_pred)
r1= accuracy_score(y_test, y_pred)
r2 = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("accuracy_score: ", r1)
print("QWK: ", r2)

accuracy_score:  0.449537750385208
QWK:  0.6825070823310198
